In [61]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from pycaret.time_series import *
import matplotlib.pyplot as plt

In [42]:
import torch
print(torch.__path__)
print(torch.__version__)
print(torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())
print("PyTorch CUDA Version:", torch.version.cuda)
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


['c:\\Users\\woutp\\Documents\\3AI\\Cloud AI\\Project\\CloudAI\\project_venv\\lib\\site-packages\\torch']
2.5.1+cpu
None
CUDA Available: False
PyTorch CUDA Version: None
Device Name: No GPU detected


In [43]:
#Subset
data = pd.read_csv(".\\price_paid_records.csv")
subset_data = data.head(25000)
subset_file_path = 'subset_25000.csv'
subset_data.to_csv(subset_file_path, index=False)

print(f"Subset of 25,000 records saved to {subset_file_path}")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [62]:
data = pd.read_csv(".\\subset_25000.csv")
data['Date of Transfer'] = pd.to_datetime(data['Date of Transfer'])
data.columns = data.columns.str.lower()
data

,transaction unique identifier,price,date of transfer,property type,old/new,duration,town/city,district,county,ppdcategory type,record status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...
24995,{435651DE-F62B-4F33-B81D-178C3E9F6FDF},53500,1995-07-11,D,N,F,DOWNHAM MARKET,KING'S LYNN AND WEST NORFOLK,NORFOLK,A,A
24996,{41DB8CBB-784C-4A06-A3A4-25D2CA1104F3},46740,1995-09-29,S,Y,F,TIPTON,SANDWELL,WEST MIDLANDS,A,A
24997,{D40E45BA-894F-4190-8A5E-25D2D1BFCC26},115000,1995-06-30,D,N,F,STOKE-ON-TRENT,STOKE-ON-TRENT,STOKE-ON-TRENT,A,A
24998,{00B2D6E2-CFAE-4E16-A701-25D2E8267D9C},240000,1995-04-03,T,N,F,LONDON,LAMBETH,GREATER LONDON,A,A


In [ ]:
# data['price_log'] = np.log1p(data['price'])
# data

In [ ]:
# data['year'] = data['date of transfer'].dt.year
# data['month'] = data['date of transfer'].dt.month
# data['quarter'] = data['date of transfer'].dt.quarter
# data

In [ ]:
# Q1 = data['price'].quantile(0.25)
# Q3 = data['price'].quantile(0.75)
# IQR = Q3 - Q1
# data = data[(data['price'] >= Q1 - 1.5 * IQR) & (data['price'] <= Q3 + 1.5 * IQR)]
# data

In [ ]:
# data[['price']] = StandardScaler().fit_transform(data[['price']])
# data

In [63]:
columns_to_encode = ['town/city', 'district', 'county', 'old/new', 'property type', 'duration', 'ppdcategory type']
le = LabelEncoder()
for col in columns_to_encode:
    data[col] = le.fit_transform(data[col])
data

,transaction unique identifier,price,date of transfer,property type,old/new,duration,town/city,district,county,ppdcategory type,record status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18,3,0,0,664,263,38,0,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09,2,0,0,383,376,94,0,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30,3,0,0,439,309,81,0,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24,3,0,0,75,240,2,0,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23,2,0,0,947,200,103,0,A
...,...,...,...,...,...,...,...,...,...,...,...
24995,{435651DE-F62B-4F33-B81D-178C3E9F6FDF},53500,1995-07-11,0,0,0,303,191,63,0,A
24996,{41DB8CBB-784C-4A06-A3A4-25D2CA1104F3},46740,1995-09-29,2,1,0,915,305,101,0,A
24997,{D40E45BA-894F-4190-8A5E-25D2D1BFCC26},115000,1995-06-30,0,0,0,864,352,89,0,A
24998,{00B2D6E2-CFAE-4E16-A701-25D2E8267D9C},240000,1995-04-03,3,0,0,553,197,37,0,A


In [64]:
data = data.drop(columns=['transaction unique identifier', 'record status - monthly file only'])
data = data.dropna()  # Handle missing values, if any
data

,price,date of transfer,property type,old/new,duration,town/city,district,county,ppdcategory type
0,25000,1995-08-18,3,0,0,664,263,38,0
1,42500,1995-08-09,2,0,0,383,376,94,0
2,45000,1995-06-30,3,0,0,439,309,81,0
3,43150,1995-11-24,3,0,0,75,240,2,0
4,18899,1995-06-23,2,0,0,947,200,103,0
...,...,...,...,...,...,...,...,...,...
24995,53500,1995-07-11,0,0,0,303,191,63,0
24996,46740,1995-09-29,2,1,0,915,305,101,0
24997,115000,1995-06-30,0,0,0,864,352,89,0
24998,240000,1995-04-03,3,0,0,553,197,37,0


In [65]:
data['year'] = pd.DatetimeIndex(data['date of transfer']).year
data['month'] = pd.DatetimeIndex(data['date of transfer']).month
data['day'] = pd.DatetimeIndex(data['date of transfer']).day
data = data.drop(columns=['date of transfer'])
data

,price,property type,old/new,duration,town/city,district,county,ppdcategory type,year,month,day
0,25000,3,0,0,664,263,38,0,1995,8,18
1,42500,2,0,0,383,376,94,0,1995,8,9
2,45000,3,0,0,439,309,81,0,1995,6,30
3,43150,3,0,0,75,240,2,0,1995,11,24
4,18899,2,0,0,947,200,103,0,1995,6,23
...,...,...,...,...,...,...,...,...,...,...,...
24995,53500,0,0,0,303,191,63,0,1995,7,11
24996,46740,2,1,0,915,305,101,0,1995,9,29
24997,115000,0,0,0,864,352,89,0,1995,6,30
24998,240000,3,0,0,553,197,37,0,1995,4,3


In [66]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)

In [67]:
cat_features = ['property type', 'old/new', 'duration', 'town/city', 'district', 'county', 'ppdcategory type']

In [ ]:
reg_setup = setup(
    data=train_data,
    fold=5,
    target='price',
    session_id=123,  # For reproducibility
    categorical_features=cat_features# Explicitly specify categorical features
)

# Train and compare models
best_model = compare_models()

# Finalize the model
final_model = finalize_model(best_model)

# Save the model
save_model(final_model, 'best_house_price_model')

print("Model training complete. The best model has been saved as 'best_house_price_model'.")

# Evaluate the performance of the best model on the test set
evaluate_model(final_model)

# Optionally, you can use predict_model to see predictions and errors
predictions = predict_model(final_model)
print(predictions)

# Pull the metrics table for the best model
metrics = pull()
print(metrics)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [71]:
# Define features including time-based features
time_based_features = cat_features + ['year', 'month', 'day']

X_test = test_data[time_based_features]
y_test = test_data['price']

# Train the model using a Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(train_data[time_based_features], train_data['price'])
y_pred_rf = rf_model.predict(X_test)

# Calculate Mean Squared Error
rf_mse = mean_squared_error(y_test, y_pred_rf)
print("Random Forest Model MSE:", rf_mse)

# Calculate Accuracy
# Convert predictions to binary outcomes
y_pred_rf_binary = [1 if pred >= 0.5 else 0 for pred in y_pred_rf]
rf_accuracy = accuracy_score(y_test, y_pred_rf_binary)
print("Random Forest Model Accuracy:", rf_accuracy)

Random Forest Model MSE: 2102997132.4847312
Random Forest Model Accuracy: 0.0
